# 2. Renaming and merging

Note: Half datasets, with separate files for east and west subplots have been merged manually in excel

In [32]:
%%time

import pandas as pd
import numpy as np
import math
import os
import datetime
from copy import copy
from pprint import pprint
import json

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# To display df nicely in loops
from IPython.display import display 
# display(df1.head()) 
# display(df2.head())

# Display rows and columns Pandas
pd.options.display.max_columns = 100
pd.set_option('display.max_rows',100)

Wall time: 0 ns


In [33]:
# Prints the current working directory
os.getcwd()
# os.listdir()

'C:\\Users\\fahad\\MegaSync\\NMBU\\GitHub\\vPheno'

## Finding Username folder to make general path for multi PC use

In [34]:
username = str(os.getcwd()).split('\\')[2]
user_path = r'C:/Users/'+username+'/'
username, user_path

('fahad', 'C:/Users/fahad/')

## Importing Data

In [35]:
main_path = r'./Data/'
path = r'./Data/renamed_merged/'
export_path = './Data/results/'

# Create export_path folder if not exists already
os.makedirs(path, exist_ok=True)
os.makedirs(export_path, exist_ok=True)

os.listdir(path)

['Graminor_2019_all.csv',
 'Masbasis_2019_all.csv',
 'Masbasis_2020_all.csv',
 'Robot_2020_all.csv']

## Data Preparation
### Creating list of complete files

In [36]:
# Get the list of all files in directory tree at given path

files_with_address = []
files_list = []

for (dirpath, dirnames, filenames) in os.walk(path):
    files_with_address += [os.path.join(dirpath, file) for file in filenames]
    files_list.extend(filenames)
    
print(len(files_with_address), 'files found in the directory')
# files_with_address
files_list

4 files found in the directory


['Graminor_2019_all.csv',
 'Masbasis_2019_all.csv',
 'Masbasis_2020_all.csv',
 'Robot_2020_all.csv']

## Data Checking/control

### Check for duplicate filenames

In [37]:
print('Total number of files are :', len(files_list))

print('Number of unique file names are:', len(set(files_list)))

print('There is/are', len(files_list) - len(set(files_list)),'duplicate file name/names.')
if len(files_list) - len(set(files_list)) > 0:
    raise NameError

Total number of files are : 4
Number of unique file names are: 4
There is/are 0 duplicate file name/names.


## Importing Genomics Data

### Checking number of unique cultivars in the field

In [38]:
# Vollebekk 2019: Graminor_2019_x_19TvPhenores_x_Vollebekk_res
# Masbasis 2020: Masbasis_x_20BMLGI1_2020_tm_x_data
# Robot 2020: Robot_x_ROBOT_2020_x_raw
# Masbasis 2019: Masbasis_2019_x_Field_data_2019

In [39]:
a_file = open(main_path+'yield_df.json', "r")
output_str = a_file.read()
# The file is imported as string

# Converting it to dictionary
output_dict = json.loads(output_str)
a_file.close()

pprint(output_dict)

{'Graminor 2019': ['Graminor_2019_x_19TvPhenores_x_Vollebekk_res',
                   '\\MegaSync\\NMBU\\Master '
                   'Thesis\\Data\\Feb2021\\Graminor_2019\\19TvPhenores.xlsx'],
 'Masbasis 2019': ['Masbasis_2019_x_Field_data_2019',
                   '\\MegaSync\\NMBU\\Master '
                   'Thesis\\Data\\Feb2021\\Masbasis_2019\\Field_data_2019.xlsx'],
 'Masbasis 2020': ['Masbasis_x_20BMLGI1_2020_tm_x_data',
                   '\\MegaSync\\NMBU\\Master '
                   'Thesis\\Data\\Feb2021\\Vollebekke-total_2020\\Masbasis\\20BMLGI1_2020_tm.xlsx'],
 'Robot 2020': ['Robot_x_ROBOT_2020_x_raw',
                '\\MegaSync\\NMBU\\Master '
                'Thesis\\Data\\Feb2021\\Vollebekke-total_2020\\Robot\\ROBOT_2020.xlsx'],
 'Staur 2019': ['Graminor_2019_x_19TvPhenores_x_Staur_res',
                '\\MegaSync\\NMBU\\Master '
                'Thesis\\Data\\Feb2021\\Graminor_2019\\19TvPhenores.xlsx']}


In [40]:
# plots_data = pd.read_excel(files_with_address[0],engine='openpyxl')
# # Pandas converts 'NA' string to NaN. Need to change those to 
# # some string to get a count as NaNs are not counted as unique values

# plots_data.Name.fillna('-', inplace=True)
# plots_data.CodeName.fillna('-', inplace=True)

# # Creating a new column as multiple plots were named 'NA' but the 
# # CodeName was different for each one of them
# plots_data['NameCode'] = plots_data.Name+plots_data.CodeName

# plots_data
# len(plots_data.NameCode.unique())
# plots_data.NameCode.value_counts()
# # plots_data.NameCode.value_counts().sum()
# # plots_data

## Importing data files to Pandas

In [41]:
   
%%time

all_df = []
for data in files_with_address:
    file_name = os.path.splitext(os.path.basename(data))[0]

    # Replce all invalid characters in the name
    file_name = file_name.replace(" ", "_")
    file_name = file_name.replace("-", "_")
    file_name = file_name.replace(")", "")
    file_name = file_name.replace("(", "")
    df_name = file_name.replace(".", "")
    # Test: Check if the same date is already present in the current dict key
    if df_name in all_df:
        print(f'A file with the same name {df_name} has already been imported. \n Please check if there is duplication of data.')
        raise NameError
    all_df.append(df_name)

    locals()[df_name] = pd.read_csv(data, index_col=False)
    print(df_name, '=====', locals()[df_name].shape)
# all_df

Graminor_2019_all ===== (601, 376)
Masbasis_2019_all ===== (528, 274)
Masbasis_2020_all ===== (688, 410)
Robot_2020_all ===== (96, 478)
Wall time: 356 ms


In [42]:
print(f'Total imported {len(all_df)}')
all_df

Total imported 4


['Graminor_2019_all',
 'Masbasis_2019_all',
 'Masbasis_2020_all',
 'Robot_2020_all']

# Simpsons Integration

## Listing down the important dates for each field

In [43]:
# Sowing date of the field
sowing = '200420'
maturity = '310720'
heading = '170620'

sowing_date = datetime.datetime.strptime(sowing, '%d%m%y').date()
maturity_date = datetime.datetime.strptime(maturity, '%d%m%y').date()
heading_date =datetime.datetime.strptime(heading, '%d%m%y').date()

In [44]:
# ToDo: Add check for duplicate columns in the df

general_col_names = ['Plot_ID', 'Blue', 'Green', 'Red', 'RedEdge', 'NIR']

# list_agg_df
yield_cols = ['GrainYield', 'Days2Heading', 'Days2Maturity']
id_cols_new = ['Plot_ID']

# Counter for location of column in columns list

# Dict for saving the name and location of the yield column/s
loc_yield_cols = {}
for df in all_df:
    loc = 0
    for cols in locals()[df].columns.tolist():
        for y_col in yield_cols:
            if not cols.find(y_col):
                loc_yield_cols[cols] = loc
                print(f'\"{cols}\" column in {df} is the yield column\n as it contains the text \"{y_col}\". It is located at location {loc}')
        loc += 1

    yield_cols_found = list(loc_yield_cols.keys())
    target_cols=yield_cols_found[0]
loc_yield_cols

"GrainYield" column in Graminor_2019_all is the yield column
 as it contains the text "GrainYield". It is located at location 6
"GrainYield" column in Masbasis_2019_all is the yield column
 as it contains the text "GrainYield". It is located at location 6
"GrainYield" column in Masbasis_2020_all is the yield column
 as it contains the text "GrainYield". It is located at location 6
"GrainYield" column in Robot_2020_all is the yield column
 as it contains the text "GrainYield". It is located at location 6


{'GrainYield': 6}

In [50]:
id_cols_new+yield_cols_found

['Plot_ID', 'GrainYield']

In [47]:
df_aoc = df[id_cols_new+yield_cols_found]
df_aoc

TypeError: string indices must be integers

In [46]:
df_aoc = df[id_cols_new+yield_cols_found]
# Renaming Grain_Yield Column #custom
df_aoc.columns = [df_aoc.columns[0]]+['GrainYield']
df_aoc.head(2)

TypeError: string indices must be integers

In [45]:
Graminor_2019_all

,Plot_ID,Blue_060619,Green_060619,Red_060619,RedEdge_060619,NIR_060619,GrainYield,NDVI_060619,MTCI_060619,DVI_060619,GDVI_060619,MTCI_CI_060619,EXG_060619,EXGR_060619,RDVI_060619,TDVI_060619,GNDVI_060619,NDRE_060619,SCCI_060619,EVI_060619,TVI_060619,VARI_060619,GARI_060619,GCI_060619,GLI_060619,NLI_060619,MNLI_060619,SAVI_060619,GSAVI_060619,OSAVI_060619,GOSAVI_060619,MSAVI2_060619,MSR_060619,GRVI_060619,WDRVI_060619,SR_060619,Blue_110619,Green_110619,Red_110619,RedEdge_110619,NIR_110619,NDVI_110619,MTCI_110619,DVI_110619,GDVI_110619,MTCI_CI_110619,EXG_110619,EXGR_110619,RDVI_110619,TDVI_110619,...,TVI_070819,VARI_070819,GARI_070819,GCI_070819,GLI_070819,NLI_070819,MNLI_070819,SAVI_070819,GSAVI_070819,OSAVI_070819,GOSAVI_070819,MSAVI2_070819,MSR_070819,GRVI_070819,WDRVI_070819,SR_070819,Blue_150819,Green_150819,Red_150819,RedEdge_150819,NIR_150819,NDVI_150819,MTCI_150819,DVI_150819,GDVI_150819,MTCI_CI_150819,EXG_150819,EXGR_150819,RDVI_150819,TDVI_150819,GNDVI_150819,NDRE_150819,SCCI_150819,EVI_150819,TVI_150819,VARI_150819,GARI_150819,GCI_150819,GLI_150819,NLI_150819,MNLI_150819,SAVI_150819,GSAVI_150819,OSAVI_150819,GOSAVI_150819,MSAVI2_150819,MSR_150819,GRVI_150819,WDRVI_150819,SR_150819
0,101,0.062427,0.129283,0.129278,0.185566,0.407937,499.624440,0.518710,3.950610,0.278659,0.278654,3.950610,0.066861,0.015155,0.380188,0.468589,0.518696,0.374675,0.722322,-0.379451,16.719722,0.000025,0.603848,2.155378,-0.138632,0.125585,0.070004,0.402991,0.402982,0.399674,0.399664,1.299123,2.592551,3.155378,-0.520277,3.155496,0.079597,0.157513,0.168775,0.220306,0.404954,0.411656,3.583244,0.236179,0.247441,3.583244,0.066653,-0.012119,0.311809,0.388215,...,12.397675,-0.448297,0.347059,9.475924,-0.379698,0.230615,0.062710,0.407691,0.450596,0.471129,0.529443,1.144776,5.724268,10.475924,-0.240062,6.128223,0.028785,0.058441,0.117516,0.140037,0.219810,0.303250,3.542136,0.102294,0.161369,3.542136,-0.029419,-0.135500,0.176127,0.188044,0.579940,0.221686,0.731033,-0.150329,3.774674,-0.401399,0.854320,2.761220,-0.333835,-0.417284,-0.155894,0.183252,0.311022,0.205689,0.368211,0.879676,1.139292,3.761220,-0.684853,1.870472
1,102,0.061705,0.129847,0.127858,0.184991,0.404994,515.532751,0.520099,3.850745,0.277136,0.275147,3.850745,0.070131,0.020976,0.379655,0.467148,0.514447,0.372896,0.716972,-0.379650,16.707685,0.010147,0.597877,2.119012,-0.132922,0.123894,0.068499,0.402481,0.398825,0.399993,0.395986,1.295456,2.605650,3.119012,-0.518888,3.167525,0.075853,0.152404,0.163355,0.220335,0.405466,0.425636,3.249073,0.242111,0.253062,3.249073,0.065600,-0.010693,0.321016,0.399166,...,11.737341,-0.482118,0.363954,9.827102,-0.392431,0.193261,0.050652,0.394101,0.441762,0.459487,0.524948,1.118907,5.473275,10.827102,-0.259012,5.885476,0.025709,0.051825,0.102399,0.122849,0.201019,0.325028,3.822563,0.098619,0.149194,3.822563,-0.024459,-0.115994,0.179037,0.184507,0.590065,0.241364,0.742593,-0.153498,3.894176,-0.393533,0.795194,2.878825,-0.329154,-0.434086,-0.144656,0.184125,0.297261,0.212809,0.361382,0.859646,1.249363,3.878825,-0.671809,1.963087
2,103,0.063729,0.132427,0.132313,0.194414,0.430182,529.501025,0.529549,3.796499,0.297869,0.297755,3.796499,0.068813,0.016002,0.397160,0.494205,0.529238,0.377472,0.712817,-0.404347,17.876711,0.000569,0.621582,2.248436,-0.138024,0.166189,0.096792,0.420523,0.420316,0.412278,0.412055,1.341230,2.696649,3.248436,-0.509292,3.251243,0.075147,0.149900,0.164103,0.221534,0.412597,0.430889,3.326803,0.248494,0.262697,3.326803,0.060550,-0.019294,0.327221,0.408071,...,12.233252,-0.350342,0.323931,9.840122,-0.338834,0.269818,0.065369,0.406650,0.437229,0.479428,0.521912,1.128248,6.655479,10.840122,-0.174221,7.032567,0.024099,0.047830,0.084905,0.113178,0.184771,0.370319,2.532189,0.099866,0.136941,2.532189,-0.013344,-0.084380,0.192308,0.190392,0.588738,0.240286,0.648862,-0.165847,4.508981,-0.341276,0.684723,2.863084,-0.298898,-0.426430,-0.123007,0.194627,0.280387,0.232422,0.348805,0.850718,1.498339,3.863084,-0.642547,2.176214
3,104,0.062525,0.127990,0.13

# Import Genomics Data

# ToDo: Dropping NAN

## Finding NAN values
### ToDo: Test: Raise error if missing values found

In [25]:
# Finding number of missing values in each dataframe
df_with_nan = []
missing_values = False
for df in all_df:
    if locals()[df].isna().sum().sum() > 0:
        print(f'Total missing values in {df} are {locals()[df].isna().sum().sum()}')
        missing_values = True
        df_with_nan.append(df)
#     if len(df_with_nan) > 0:
#         raise ValueError
if not missing_values:
    print('No missing value found in any dataframe')

No missing value found in any dataframe


In [26]:
Graminor_2019_all.isnull().sum().sort_values()

NameError: name 'Graminor_2019_all' is not defined

In [27]:
df_with_nan

[]

In [28]:
# Finding which column has NAN values
for df in df_with_nan:
    print(f'{df}:\n {locals()[df].shape[1]-locals()[df].dropna(axis=1).shape[1]} columns or {locals()[df].shape[0]-locals()[df].dropna().shape[0]} rows to be dropped,')

## ToDo: Automate: Drop rows with missing values in df_with_nan

In [29]:
print(f'{Graminor_eastwest_020719_NIR_half_missing.shape} Before dropping')
# Graminor_eastwest_020719_NIR_half_missing.dropna(inplace=True)
print(f'{Graminor_eastwest_020719_NIR_half_missing.shape} After dropping')


NameError: name 'Graminor_eastwest_020719_NIR_half_missing' is not defined

## ORRR

## ToDo: Droppping df with Nan from the all_df_std

In [30]:
print(f'Number of items in all_df is {len(all_df)}')

Number of items in all_df is 0


In [31]:
# for df in df_with_nan:
#     all_df.remove(df)

###  ToDo: Update field_year_dict and sorted_field_year_dict after dropping the dataset

In [32]:
print(f'Number of items in all_df now is {len(all_df)}')

Number of items in all_df now is 0


# Data Trends

## Normal Distribution of data

ToDo:  
see the distribution of data if it is normal  
else make transpose to make it normal  
dist in Gausion function   
in each field  
what if the data is normal dist?  
the use some transpose to box pox   
try diff funct to see which one iis able to make data normal  
make heat map of whole if not normal  
see which parts are not normal and exculde them  
ls_means in R to make the normalisation/transpose  
pearson corr bw yield and indices for diff dates  


In [33]:
x_labels

NameError: name 'x_labels' is not defined

### Yeo-Johnson Transformation

In [34]:
col_for_plotting = ['Blue', 'Green', 'Red', 'RedEdge', 'NIR', 'NDVI', 'MTCI', 'EVI']

from sklearn.preprocessing import PowerTransformer, normalize, StandardScaler
data_agg_list = ['_median_indices']

# col_for_plotting = ['Blue']
# col_for_plotting = ['Green']
# col_for_plotting = ['Red']

for d_type in data_agg_list:
    for col in col_for_plotting:
        fields = len(field_year_dict_yield.keys())
        rows = math.ceil(fields/2)
        
        fig, ax = plt.subplots(rows,2, figsize=(15,10))
        plots = ax.flatten()
        n = 0
        # TODO: Fix the x ticks
        

        for field_sample, dates in sorted_field_year_dict_yield.items():
            x_labels = []
            # Adding required data to a temp dataframe
            temp_df = pd.DataFrame()
            for date in dates:
                date_str = date.strftime('%d%m%y')
                field_df = field_sample[:-5]+'_'+date_str+d_type
                temp_df[date] = locals()[field_df][col]
                x_label = date.strftime('%d-%m-%y')+':'+str(len(locals()[field_df][col]))
                
                x_labels.append(x_label)
                x_labels= list(set(x_labels))
            # Transform the df
#             pt = PowerTransformer(method='box-cox', standardize=False)
            pt = PowerTransformer(method='yeo-johnson', standardize=False)

            temp_arr = pt.fit_transform(temp_df)
            temp_df = pd.DataFrame(temp_arr)
            
            # Adding field plot to the subplots
            num_of_fields = len(field_year_dict_yield.keys())
            
            text = "Grain Yield"
            ax_n = plots[n]
            
            temp_df.boxplot(ax=ax_n)
            ax_n.set_xticklabels(x_labels, rotation=-35)
            ax_n.set_title(field_sample+'_'+col+d_type[:-5]+'_yeo-johnson')
            
#             # Printing the grain yield in plot of the fiels_sample for reference
#             ax_n.text(0.85, 1.05, text, ha='center', va='top', weight='bold', color='blue', transform=ax_n.transAxes)
            n+=1
        plt.tight_layout()



NameError: name 'field_year_dict_yield' is not defined

### Box-Cox Transformation

In [35]:
col_for_plotting = ['Blue', 'Green', 'Red', 'RedEdge', 'NIR', 'NDVI', 'MTCI', 'EVI']

from sklearn.preprocessing import PowerTransformer, normalize, StandardScaler
data_agg_list = ['_median_indices']

# col_for_plotting = ['Blue']
# col_for_plotting = ['Green']
# col_for_plotting = ['Red']

for d_type in data_agg_list:
    for col in col_for_plotting:
        fields = len(field_year_dict_yield.keys())
        rows = math.ceil(fields/2)
        
        fig, ax = plt.subplots(rows,2, figsize=(15,10))
        plots = ax.flatten()
        n = 0
        # TODO: Fix the x ticks
        for field_sample, dates in sorted_field_year_dict_yield.items():
            
            # Adding required data to a temp dataframe
            temp_df = pd.DataFrame()
            for date in dates:
                date_str = date.strftime('%d%m%y')
                field_df = field_sample[:-5]+'_'+date_str+d_type
                temp_df[date] = locals()[field_df][col]
            x_labels = temp_df.columns.tolist()

            # Transform the df
#             pt = PowerTransformer(method='box-cox', standardize=False)
            pt = PowerTransformer(method='box-cox', standardize=False)

            # Taking absolute values of the dataframe(avoiding negative values)
            temp_arr = pt.fit_transform(temp_df.abs())
            temp_df = pd.DataFrame(temp_arr)
            
            # Adding field plot to the subplots
            num_of_fields = len(field_year_dict_yield.keys())
            
            text = "Grain Yield"
            ax_n = plots[n]

            temp_df.boxplot(ax=ax_n)
            ax_n.set_xticklabels(x_labels, rotation=90)
            ax_n.set_title(field_sample+'_'+col+d_type[:-5]+'_box-cox')
            
#             # Printing the grain yield in plot of the fiels_sample for reference
#             ax_n.text(0.85, 1.05, text, ha='center', va='top', weight='bold', color='blue', transform=ax_n.transAxes)
            n+=1
        plt.tight_layout()



NameError: name 'field_year_dict_yield' is not defined

### ToDo: Identify Dates and index with problems

### Ecxclude the problematic data/dates
or
### Take average values where the problematic data is

Take average of data for date 20200708 and 20200624  
Masbasis  
Cleanup  
Remove dates which have drop  

## ToDo: Remove outliers

### Find AUC for all dates of one field
See if it covers tha gaps under the dates,i.e.

Since data points are different  
Flying time is different  
Cover the gaps between the dates  

Since the data collection is not uniform throughout the year so AUC will give a single value instead of multiple values for one field year which will be representative of all the dates 

#### Option 1: Use Scipy

In [36]:
import scipy
scipy.__version__

'1.6.1'

In [37]:
from scipy import integrate
from scipy.integrate import simps

In [38]:
from scipy.integrate import simpson

In [39]:
x = np.arange(0, 10)
y = np.arange(0, 10)


In [40]:
# integrate.simpson(y, x)
integrate.simps(y, x)

40.5

In [41]:
y = np.power(x, 3)
y

array([  0,   1,   8,  27,  64, 125, 216, 343, 512, 729], dtype=int32)

In [42]:
integrate.simpson(y, x)
# integrate.simps(y, x)


1642.5

In [43]:
integrate.quad(lambda x: x**3, 0, 9)[0]

1640.25

In [44]:
integrate.simpson(y, x, even='first')
# integrate.simps(y, x, even='first')

1644.5

#### Option 2

In [45]:
data
# plot: Plot ID
# x: Number of days after sowing or actual date
# y: Value of the index


NameError: name 'data' is not defined

In [46]:
# x: Days from sowing to data collection
# May 5 2019 Masbasis and Graminor
# Robot: 

data={'plot':['1','1','2','2','3','3'],'x':['5','6','7','8','9','10'],'y':['0.9','0.8','0.7','0.6','0.5','0.4'] }

ACC=[]
A=pd.DataFrame(data, columns=['plot','x','y'])
AA=0

for item in range(len(A)-1):
    if A['plot'][item]== A['plot'][item+1]:
        Ans=(float((A['y'][item]))+float((A['y'][item+1])))*((float((A['x'][item+1]))-float((A['x'][item]))))/2
        AA+=Ans
        print(AA)
        ACC.append(AA)

0.8500000000000001
1.5
1.95


### Alternative

In [47]:
df1=Data.set_index(['Plot'])
ACC=[]

for item in Numbers_final:
    df2=df1[df1.index==item]
    df2=df2.filter(['Blue', 'Green', 'Red', 'RedEdge', 'NIR','NDVI', 'MTCI', 'EVI', 'DVI', 'RVI', 'VARI', 'EXG', 'EXGR', 'GLI', 'GNDVI', 'GVI','Time','timepoint'], axis=1)
    df2=df2.sort_values(by='timepoint')
    df3=df2.reset_index()

AA=0
for j in range(0,3):
    Ans=(float((df3['GVI'][j]))+float((df3['GVI'][j+1])))*((float((df3['timepoint'][j+1]))-float((df3['timepoint'][j]))))/2
    AA+=Ans

    print(AA)
    ACC.append(AA)



DA=pd.DataFrame(ACC)
DD=pd.DataFrame(Numbers_final)
DDA=pd.concat([DD, DA], axis=1)
DDA.to_excel('Staur_Accumulative_GVI_2019.xlsx')

NameError: name 'Data' is not defined

### Time series data vs the AUC

# ToDo: Model Training


Make model for one year at a time and try to predict yield of another field  

TODO: Train on Masbasis 2019 an 2020  
Test on Staur  

Use data until august for yield prediction since it is most relavant  
Use all data for predicting date to maturity  

Data Collection:  
Data collection usually starts after heading  
2019 has the data before hading as well. To use that, dont use dates before heading  

NDVI is resistant to shadows  

DAT390 Report: Do the report with Robot Data only  

TODO: Use AUC for each index for prediction  

TODO:   
Time series data vs the AUC  